In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pickle as pkl
from astropy.io import fits
import scipy.sparse
import scipy.linalg

r0 = 2.818e-13 # classical electron radius in cm
m_e = 9.10938e-28 # electron mass in gram
c = 2.998e10 # speed of light in cm/s
h = 6.6261e-27 # planck constant in cm^2 g s-1
eV_to_erg = 1.6022e-12
parsec_to_cm = 3.085677581e18 # cm per parsec

k = 1.3808e-16 # boltzmann constant in cm^2 g s^-2 K^-1
T_CMB = 2.725 # CMB temperature today in K

H_0 = 2.184e-18 # current hubble constant in s^-1
H_r = 3.24076e-18 # 100 km/s/Mpc to 1/s
h_0 = H_0 / H_r
omega_b_0 = 0.0224 / h_0 / h_0  # current baryon abundance
m_p = 1.67262193269e-24 # proton mass in g
G = 6.6743e-8 # gravitational constant in cm^3 g^-1 s^-2
f_He = 0.079 # helium mass fraction
f_H = 0.76 # hydrogen fraction
Y_He = 0.24

E_e = m_e * c * c
rho_crit = 3 * H_0 * H_0 / 8 / np.pi / G
e = 2.718281828459 # base of natural log

z = 2
z2 = (1 + z) / 3

In [2]:
with open('rate_10Mpc.pkl', 'rb') as f:
    rata_arr = pkl.load(f)
    
rate_arr = np.array(rata_arr)
rate_arr = rate_arr.reshape(400,400) # [i][j]: gamma[i] theta[j], rate_arr[-1] = nan
rate_trans = np.transpose(rate_arr) # [i][j]: theta[i] gamma[j], rate_trans[i][-1] = nan

gamma_e_arr = np.logspace(8, 14, 400) * eV_to_erg / m_e / c**2 # lab-frame Lorentz factor of the electron produced
theta_e_arr = np.logspace(-8,0, 400)

In [3]:
def compton_f(x):
    if x > 1e-2:
        return ((-24 - 144*x - 290*x**2 - 194*x**3 + 20*x**4 + 28*x**5) / 3 / x**2 / (1+2*x)**3 + (4+4*x-x**2) / x**3 * np.log(1+2*x))
    return 8/5*x**2 - 32/5*x**3 + 2248/105*x**4 - 1376/21*x**5 + 11840/63*x**6

def compton_FF(x):
    n = 1024
    y = np.linspace(x / (2 * n), x - x / (2 * n), n)
    result = 0.
    for i in range(n): result += compton_f(y[i]) * y[i]
    result *= x / n
    return result

In [4]:
jtable = np.loadtxt('EBL_KS18_Q20_z_ 2.0.txt')
nstep = np.shape(jtable)[0] - 1
jtable[:,0] = 2.99792458e18 / jtable[:,0] # convert to Hz
nu = (jtable[:-1,0] + jtable[1:,0]) / 2
dnu = (jtable[:-1,0] - jtable[1:,0])
JnuEBL = (jtable[:-1,1] + jtable[1:,1]) / 2

xx = h*nu/k/T_CMB
Bnu = 2 * h * nu**3 / c**2 / np.expm1(xx) ### it gives RuntimeWarning: overflow encountered in expm1, is it ok?
Jnu = JnuEBL + Bnu

/var/folders/k2/qxng019s7y17hp_kgkzy9b180000gn/T/ipykernel_45131/3605387613.py:9: RuntimeWarning: overflow encountered in expm1
  Bnu = 2 * h * nu**3 / c**2 / np.expm1(xx) ### it gives RuntimeWarning: overflow encountered in expm1, is it ok?


In [5]:
def sumsq(jp, j): # theta_rms from gamma_jp to gamma_j
    sumsq = 0
    for i in range(jp, j-1, -1):
        all_int = 0
        factors = Bnu / nu**3 * dnu
        for k in range(nstep):
            all_int += compton_FF(2 * gamma_e_arr[i] * h * nu[k] / m_e / c**2) * factors[k]
            
        tp2 = 135 * m_e * c**2 / 128 / np.pi**4 / gamma_e_arr[i]**3 * (m_e * c**2 / k / T_CMB)**4 * all_int
        sumi = tp2 / (m_e * c * gamma_e_arr[i])**2 * np.log(gamma_e_arr[1] / gamma_e_arr[0])
        sumsq += sumi
        
    return sumsq**.5

In [6]:
def calculate_solid_angles(theta):
    # theta_half[i] = theta[i-1/2]
    theta_half = (theta[1:] + theta[:-1]) / 2
    theta_half = np.concatenate([[theta[0]/2], theta_half, [3*theta[-1]/2 - theta[-2]/2]])

    Omega = 2 * np.pi * (theta_half[2:]**2 - theta_half[:-2]**2) / 2  # solid angles of annulus
    Omega = np.concatenate([Omega, [2 * np.pi * (theta[-1]**2 - theta_half[-2]**2)]])  # add the last circle

    return Omega, theta_half

In [7]:
Omega, theta_half = calculate_solid_angles(theta_e_arr)

In [8]:
### I'm not sure if the tau=1 smoothed Fourier transform looks right, when I apply it, it seems to give me nan

def smoothed_Fourier_transform(f, tau = 1):
    # compute the Fourier transform
    F = np.fft.fft(f)
    
    # define the Gaussian window function
    n = len(f)
    t = np.arange(n)
    window = np.exp(-0.5 * ((t - n/2) / tau)**2)
    
    # apply the smoothing operation using the Gaussian window
    F1 = F*window
    # take the inverse Fourier Transform to obtain the smoothed Fourier Transform
    F1 = np.fft.ifft(F1)
    
    return F1

This is my thinking of building M matrix, but I'm not sure if it looks right as you say we don't need to calculate
f1 explicitly

1: $\sum_{i=0}^{x-1} \sum_j \Omega[i]M[i,j]f_1[j] = Y[x-1]$

2: $\sum_{i=0}^x \sum_j \Omega[i]M[i,j]f_1[j] = Y[x]$

2-1: $\sum_j \Omega[x]M[x,j]f_1[j] = Y[x] - Y[x-1]$

$\Omega[x]$ isn't depend on j, so can be taken out, then

$\sum_jM[x,j]f_1[j] = (Y[x]-Y[x-1])/\Omega[x]$

$\sum_jM[x,j]f_1[j] = dot(f_1, M[x].T) = dot(M[x], f_1.T)$

In [9]:
def build_M(f, theta, Omega, theta_half):
    N = len(f)
    M = np.zeros((N, N))
    Y = np.zeros((N, )) # the right-hand-side of the sum for each imax
    
    f1 = smoothed_Fourier_transform(f)
    #print('f1', f1)
    
    for i in range(1, N): # i = imax, imax is [1, N]
        Y[i] = -2 * np.pi * theta_half[i] * (f1[i] - f1[i-1]) / (theta[i] - theta[i-1])
        
    for i in range(1, N):
        rhs = (Y[i] - Y[i-1]) / Omega[i]
        M[i] = (rhs / f1 / N).reshape(1, N)
    
    for j in range(N):
        sum_OM = 0
        for i in range(1, N):
            sum_OM += Omega[i] * M[i][j]
        M[0][j] = -sum_OM / Omega[0] # sum_i Omega[i]*M[i][j] = 0 for all j
    
    return M

$n_{ij}(t) = 1/(\Gamma_{IC,0}\ \gamma_j^2) ∑_{j’=j}^{j’_{max}} ∆gamma_{j’} ∑_{i’} [exp(theta_{rms, j’->j}^2/4*M)]_{ii’} S_{i’j’}$

But I think I understand something wrong when build get_nij function, because by doing this, both f1 and M give me
all nan values

In [10]:
def get_nij(i, j, t): # nij = dN / (dV dgammae_j dOmegae_i)
    Gamma_IC0 = 1.1e-18 * z2**4
    gamma_max_inv = 1. / gamma_e_arr[j] - Gamma_IC0 * t
    if gamma_max_inv>1e-99:
        gamma_max = 1./gamma_max_inv
    else:
        gamma_max = 1e99
        
    nij = 0
    
    for jp in range(j, 399):
        if gamma_e_arr[jp] <= gamma_max:
            theta_rms = sumsq(jp, j)
            M = build_M(rate_trans[i], theta_e_arr, Omega, theta_half)
            print(M)
            M_expm = scipy.linalg.expm(theta_rms**2/4 * M)
            sum_MS = np.dot(M[i], rate_trans[:, jp])
                
            sum_ij = (gamma_e_arr[jp] - gamma_e_arr[jp-1]) * sum_MS
            
            if gamma_e_arr[jp+1] > gamma_max:
                fraction = (gamma_max - gamma_e_arr[jp]) / (gamma_e_arr[jp+1] - gamma_e_arr[jp])
                sum_ij = sum_ij * fraction
                if i==0 and j%50==25: print(t, j, jp, fraction)
            nij += sum_ij
    nij = nij / (Gamma_IC0 * gamma_e_arr[j]**2)
    return nij

In [11]:
t_arr = [1e8, 5e8, 1e9, 5e9, 1e10, 5e10, 1e11, 5e11, 1e12, 5e12, 1e13]
t_num = len(t_arr)

In [12]:
nij = np.zeros((t_num, 400, 400)) # t theta[i] gamma[j]
for k in range(t_num):
    print('time', k, t_arr[k])
    for i in range(400):
        for j in range(1, 400):
            nij[k][i][j-1] = get_nij(i, j, t_arr[k])

time 0 100000000.0


/var/folders/k2/qxng019s7y17hp_kgkzy9b180000gn/T/ipykernel_45131/2048202193.py:10: ComplexWarning: Casting complex values to real discards the imaginary part
  Y[i] = -2 * np.pi * theta_half[i] * (f1[i] - f1[i-1]) / (theta[i] - theta[i-1])
/var/folders/k2/qxng019s7y17hp_kgkzy9b180000gn/T/ipykernel_45131/2048202193.py:14: ComplexWarning: Casting complex values to real discards the imaginary part
  M[i] = (rhs / f1 / N).reshape(1, N)


[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]


ValueError: cannot convert float NaN to integer